In [8]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as RMSE

df = pd.read_csv('train.csv')
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


### Preprocess categorical columns

In [10]:
imputer = SimpleImputer(strategy='mean')
df['engine_volumn'] = df["engine"].str.extract(r"(\d+\.?\d*)L").astype('float')

df['engine_volumn'] = imputer.fit_transform(df[['engine_volumn']])
df['engine_volumn'].isna().sum()

0

In [3]:
df['transmission'] = df['transmission'].str.lower()

# Remove leading/trailing whitespaces
df['transmission'] = df['transmission'].str.strip()

# Handle different representations of the same transmission type
transmission_mapping = {
    'a/t': 'automatic',
    'auto': 'automatic',
    'manual': 'manual',
    'automatic cvt': 'cvt',
    'cvt transmission': 'cvt',
    'transmission w/dual shift mode': 'automatic',
    'automatic, 8-spd pdk dual-clutch': 'automatic',
    'automatic, 8-spd m steptronic w/drivelogic, sport & manual modes': 'automatic',
    'automatic, 8-spd dual-clutch': 'automatic',
    'auto, 6-spd w/cmdshft': 'automatic',
    'automatic, 7-spd s tronic dual-clutch': 'automatic',
    'manual, 6-spd': 'manual'
}

df['transmission'].replace(transmission_mapping, inplace=True)

# Group similar transmission types together
def group_transmission(transmission):
    if 'automatic' in transmission:
        return 'automatic'
    elif 'manual' in transmission:
        return 'manual'
    elif 'cvt' in transmission:
        return 'cvt'
    else:
        return 'other'

df['transmission'] = df['transmission'].apply(group_transmission)

df['transmission']

C:\Users\voqua\AppData\Local\Temp\ipykernel_14300\1866632910.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transmission'].replace(transmission_mapping, inplace=True)


0            other
1            other
2            other
3        automatic
4        automatic
           ...    
54268        other
54269        other
54270    automatic
54271    automatic
54272        other
Name: transmission, Length: 54273, dtype: object

In [11]:
# Standardize exterior color values
df['ext_col'] = df['ext_col'].str.lower()
print(df['ext_col'].value_counts())


ext_col
black                      15078
white                      13422
gray                        7909
silver                      5161
blue                        4668
                           ...  
balloon white                  1
firenze red metallic           1
orca black metallic            1
radiant red metallic ii        1
magnetic gray clearcoat        1
Name: count, Length: 260, dtype: int64


In [12]:
mapping = {
    "At least 1 accident or damage reported": True,
    "None reported": False
}

# Apply mapping to the column
df['accident'] = df['accident'].map(mapping)

df['clean_title'] = df['clean_title'].apply(lambda x: True if x == 'Yes' else False)
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,engine_volumn
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,blue,Gray,False,True,11000,3.5
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,black,Black,False,True,8250,3.0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,purple,Beige,False,True,15000,4.2
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,gray,Brown,False,True,63500,3.0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,white,Black,False,True,7850,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,white,Brown,False,True,29000,4.4
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,black,Black,True,True,6500,2.0
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,white,Black,True,True,18950,3.6
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,beige,Brown,False,True,194965,4.0


In [13]:
df = df.drop(['engine','model'],axis =1)

df_dummies = pd.get_dummies(df[['brand','fuel_type','transmission','ext_col','int_col']])

object_columns = df.select_dtypes(include=['object'])
object_columns

df_num = df.drop(object_columns,axis = 1)
x_num = df_num[['model_year','milage','engine_volumn']]
x_cat = pd.concat([df_dummies, df[['accident', 'clean_title']]],axis = 1).values
y = df_num['price'].values

In [14]:
scale = StandardScaler()
x_num_scaled = scale.fit_transform(x_num.values)
x_train = np.concatenate((x_num_scaled,x_cat), axis = 1)
print(x_train.shape,y.shape)

(54273, 495) (54273,)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(x_train,y, test_size = 0.3, random_state =5)

gbt = GradientBoostingRegressor(n_estimators = 10,
                            max_depth = 10,
                            random_state = 5)
gbt.fit(X_train,y_train)
y_pred = gbt.predict(X_test)
rmse = RMSE(y_test, y_pred)
print (rmse)

75453.1917178881


In [24]:
test_df = pd.read_csv('test.csv')


test_df['engine_volumn'] = test_df["engine"].str.extract(r"(\d+\.?\d*)L").astype('float')
test_df['engine_volumn'] = imputer.fit_transform(test_df[['engine_volumn']])
test_df['engine_volumn'].isna().sum()

def categorize_transmission(trans):
    trans = trans.lower().strip()  # Convert to lowercase and strip whitespaces
    if 'auto' in trans or 'a/t' in trans or 'automatic' in trans and "cvt" not in trans:
        return 'automatic'
    elif 'cvt' in trans:
        return 'cvt'
    elif 'manual' in trans:
        return 'manual'
    else:
        return 'other'

# Apply the categorization function to the 'transmission' column
test_df['transmission'] = test_df['transmission'].apply(categorize_transmission)
test_df['transmission'].unique()

# Standardize exterior color values
test_df['ext_col'] = test_df['ext_col'].str.lower()

# Remove leading/trailing whitespaces
test_df['ext_col'] = test_df['ext_col'].str.strip()
print(test_df['ext_col'])

test_df = test_df.drop(['engine','model'],axis =1)
mapping = {
    "At least 1 accident or damage reported": True,
    "None reported": False
}
# Apply mapping to the column
test_df['accident'] = test_df['accident'].map(mapping)
test_df['clean_title'] = test_df['clean_title'].apply(lambda x: True if x == 'Yes' else False)


0         white
1        silver
2          blue
3         white
4         white
          ...  
36178      gray
36179       red
36180     beige
36181     black
36182     black
Name: ext_col, Length: 36183, dtype: object


In [25]:
test_dummies = pd.get_dummies(test_df[['brand','fuel_type','transmission','ext_col','int_col']])
#Identify missing columns
missing_columns = [col for col in df_dummies.columns if col not in test_dummies.columns]
# Add missing columns to test_dummies with value = False
for col in missing_columns:
    test_dummies[col] = False

# Ensure the columns are in the same order as df_dummies
test_dummies = test_dummies[df_dummies.columns]
object_columns = test_df.select_dtypes(include=['object'])
test_num = test_df.drop(object_columns,axis = 1)

xtest_num = test_num[['model_year','milage','engine_volumn']]
xtest_num_scaled = scale.fit_transform(xtest_num.values)
xtest_cat = pd.concat([test_dummies, test_df[['accident', 'clean_title']]],axis = 1).values
xtest_f = np.concatenate((xtest_num_scaled, xtest_cat), axis = 1)

price_pred = gbt.predict(xtest_f)

C:\Users\voqua\AppData\Local\Temp\ipykernel_14300\2964543728.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dummies[col] = False
C:\Users\voqua\AppData\Local\Temp\ipykernel_14300\2964543728.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dummies[col] = False
C:\Users\voqua\AppData\Local\Temp\ipykernel_14300\2964543728.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

In [27]:
sub = pd.DataFrame({'id':test_df['id'],
                    'price':price_pred})
sub

,id,price
0,54273,29077.410458
1,54274,26328.311867
2,54275,27744.213751
3,54276,53472.186999
4,54277,42462.191054
...,...,...
36178,90451,60982.547053
36179,90452,21205.436190
36180,90453,21394.475560
36181,90454,45551.823247


In [28]:
sub.to_csv('submission.csv', index=False)